In [ ]:
#TEIC-module 3.0 (expert model)

filepath1 ="./dataset/TEIC_data(expert_non-ICU)(TEIC-table3).csv" #filepath to the dataset (TEIC-table3)
filepath2 ="." #filepath to the folder where to save the training dataset (expert data)
filepath3 ="." #filepath to the folder where to save the testing dataset (expert data)


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

    
data = pd.read_csv(filepath1)
ID = data[["index"]]
parameter_list = ["Age","body weight","BMI","Creatinine clearance","Alb","T1","T2","T3","T4","T5","T6","T7","TAM","TPM"]
parameter = data[parameter_list]
display(parameter)
loading = data[["loading dose"]]#experts' loading dosing
maintenance = data[["maintenance dose"]] #exloading_dose = np.copy(loading)perts' maintenance dosing
TDM = data[["TDM","TDM exclusion criteria"]]


#random state and train_test_split
rs = 6 # (0-5, 6-11, 12-17, 18-23) 1, 4, 6 (0-11, 12-23) 4, 5, 6
ID_train, ID_test, parameter_train, parameter_test, loading_train, loading_test, maintenance_train, maintenance_test, TDM_train, TDM_test = train_test_split(ID,parameter, loading, maintenance,TDM, test_size=0.2,random_state=rs)
train = pd.concat([ID_train,parameter_train,loading_train,maintenance_train,TDM_train],axis=1).set_index("index")
test = pd.concat([ID_test,parameter_test,loading_test,maintenance_test,TDM_test],axis=1).set_index("index")
display(train)
display(test)
train.to_csv(str(filepath2)+"/TEIC_non-ICU_train_rs{}(TEIC-table19).csv".format(rs), index=True)    
test.to_csv(str(filepath3)+"/TEIC_non-ICU_test_rs{}(TEIC-table20).csv".format(rs), index=True)      


#GridSearchCV
grid_rfc = {"n_estimators":[10,20,40,80,160,320],"max_depth":[2,4,8,16,32,64]}
grid_search_loading_rfc = GridSearchCV(RandomForestClassifier(random_state=rs), grid_rfc,cv=5)
grid_search_maintenance_rfc = GridSearchCV(RandomForestClassifier(random_state=rs), grid_rfc,cv=5)

grid_search_loading_rfc.fit(parameter_train, loading_train)
grid_search_maintenance_rfc.fit(parameter_train, maintenance_train)

#training phase
rfc_loading_train_score = grid_search_loading_rfc.score(parameter_train,loading_train) #score
rfc_maintenance_train_score = grid_search_maintenance_rfc.score(parameter_train,maintenance_train) #score

#testing phase
rfc_loading_test_score = grid_search_loading_rfc.score(parameter_test,loading_test) #score
rfc_maintenance_test_score = grid_search_maintenance_rfc.score(parameter_test,maintenance_test) #score
rfc_loading_params = grid_search_loading_rfc.best_params_ #best parameter
rfc_maintenance_params = grid_search_maintenance_rfc.best_params_ #best parameter

#show best parameters and scores
print("train score for loading dose is {}".format(rfc_loading_train_score))
print("train score for maiantenance dose is {}".format(rfc_maintenance_train_score))

print("test score for loading dose is {}".format(rfc_loading_test_score))
print("test parameters for loading dose is {}".format(rfc_loading_params))

print("test score for maiantenance dose is {}".format(rfc_maintenance_test_score))
print("test parameters for maintenance dose is {}".format(rfc_maintenance_params))

test["loading dose (ML)"] = grid_search_loading_rfc.predict(parameter_test)
test["maintenance dose (ML)"] = grid_search_maintenance_rfc.predict(parameter_test)
display(test)
test.to_csv(str(filepath3)+"/TEIC_result_(non-ICU_expertML)_internal-validation_rs{}(TEIC-table21).csv".format(rs), index=True)  

#show feature importance
print("feature importance for loading dose is {}".format(grid_search_loading_rfc.best_estimator_.feature_importances_))
print("feature importance for maintenance dose is {}".format(grid_search_maintenance_rfc.best_estimator_.feature_importances_))


,Age,body weight,BMI,Creatinine clearance,Alb,T1,T2,T3,T4,T5,T6,T7,TAM,TPM
0,79,44.40,20.519068,43.208333,2.8,0,0,1,0,0,0,0,1,0
1,99,48.70,24.357356,21.625828,3.1,0,1,0,0,0,0,0,1,0
2,86,56.00,21.207101,23.595506,3.8,1,0,0,0,0,0,0,0,1
3,41,65.50,23.429809,117.774039,3.1,0,0,0,0,1,0,0,1,0
4,51,55.70,21.408511,20.180580,1.6,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,54,85.30,30.622198,103.965419,3.3,0,1,0,0,0,0,0,1,0
114,44,78.05,31.068077,90.261905,3.5,0,1,0,0,0,0,0,0,1
115,84,54.20,20.399714,45.821256,2.2,1,0,0,0,0,0,0,1,0
116,81,42.90,19.532637,34.745397,2.4,0,0,0,1,0,0,0,0,1


,Age,body weight,BMI,Creatinine clearance,Alb,T1,T2,T3,T4,T5,T6,T7,TAM,TPM,loading dose,maintenance dose,TDM,TDM exclusion criteria
index,,,,,,,,,,,,,,,,,,
81,84,41.90,16.084478,42.323232,2.2,0,1,0,0,0,0,0,1,0,400 mg 3 doses,0 mg,21.2,0
76,92,55.50,20.360987,31.092437,2.9,0,0,0,1,0,0,0,0,1,500 mg 4 doses,200 mg,17.9,0
92,71,56.75,20.844812,14.089486,1.8,0,1,0,0,0,0,0,1,0,400 mg 3 doses,0 mg,16.9,0
17,30,45.00,18.186749,110.887097,3.8,0,1,0,0,0,0,0,1,0,400 mg 4 doses,400 mg,14.3,0
20,52,43.85,19.619467,42.180813,2.6,1,0,0,0,0,0,0,0,1,400 mg 4 doses,0 mg,23.3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,33,44.10,18.619214,121.101902,2.3,0,0,1,0,0,0,0,1,0,400 mg 4 doses,0 mg,22,4
116,68,66.40,23.220997,62.641509,3.1,0,0,0,0,1,0,0,1,0,600 mg 4 doses,0 mg,16,0
135,55,52.40,17.508103,167.192192,2.5,1,0,0,0,0,0,0,0,1,600 mg 3 doses,400 mg,18.2,0


,Age,body weight,BMI,Creatinine clearance,Alb,T1,T2,T3,T4,T5,T6,T7,TAM,TPM,loading dose,maintenance dose,TDM,TDM exclusion criteria
index,,,,,,,,,,,,,,,,,,
94,66,57.80,22.105880,56.042977,2.0,1,0,0,0,0,0,0,0,1,400 mg 4 doses,400 mg,none,0
19,80,49.80,20.152283,37.930108,3.4,0,0,0,1,0,0,0,1,0,400 mg 4 doses,200 mg,15.9,0
1,79,44.40,20.519068,43.208333,2.8,0,0,1,0,0,0,0,1,0,400 mg 4 doses,0 mg,20.9,0
142,74,40.60,18.189671,15.660479,3.0,0,1,0,0,0,0,0,1,0,400 mg 4 doses,200 mg,34.1,0
24,64,55.00,19.486961,29.772080,3.5,0,0,1,0,0,0,0,1,0,600 mg 3 doses,0 mg,21.2,0
42,25,46.35,16.699357,149.825149,2.9,0,1,0,0,0,0,0,1,0,400 mg 4 doses,400 mg,16.7,0
134,75,54.10,19.398329,74.000421,2.1,0,0,1,0,0,0,0,1,0,600 mg 3 doses,0 mg,20.3,0
27,77,73.50,27.494880,43.750000,2.8,0,0,0,0,1,0,0,0,1,800 mg 3 doses,300 mg,26,0
82,84,37.50,14.889407,34.722222,2.1,0,0,0,1,0,0,0,1,0,400 mg 3 doses,400 mg,31.1,0


/Users/tetsuomatsuzaki/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
/Users/tetsuomatsuzaki/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/tetsuomatsuzaki/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/tetsuomatsuzaki/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:598: DataCo

train score for loading dose is 0.7340425531914894
train score for maiantenance dose is 0.9893617021276596
test score for loading dose is 0.4583333333333333
test parameters for loading dose is {'max_depth': 4, 'n_estimators': 20}
test score for maiantenance dose is 0.6666666666666666
test parameters for maintenance dose is {'max_depth': 8, 'n_estimators': 40}


/Users/tetsuomatsuzaki/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_search.py:880: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


,Age,body weight,BMI,Creatinine clearance,Alb,T1,T2,T3,T4,T5,T6,T7,TAM,TPM,loading dose,maintenance dose,TDM,TDM exclusion criteria,loading dose (ML),maintenance dose (ML)
index,,,,,,,,,,,,,,,,,,,,
94,66,57.80,22.105880,56.042977,2.0,1,0,0,0,0,0,0,0,1,400 mg 4 doses,400 mg,none,0,600 mg 4 doses,400 mg
19,80,49.80,20.152283,37.930108,3.4,0,0,0,1,0,0,0,1,0,400 mg 4 doses,200 mg,15.9,0,400 mg 4 doses,200 mg
1,79,44.40,20.519068,43.208333,2.8,0,0,1,0,0,0,0,1,0,400 mg 4 doses,0 mg,20.9,0,400 mg 3 doses,0 mg
142,74,40.60,18.189671,15.660479,3.0,0,1,0,0,0,0,0,1,0,400 mg 4 doses,200 mg,34.1,0,400 mg 4 doses,200 mg
24,64,55.00,19.486961,29.772080,3.5,0,0,1,0,0,0,0,1,0,600 mg 3 doses,0 mg,21.2,0,400 mg 3 doses,0 mg
42,25,46.35,16.699357,149.825149,2.9,0,1,0,0,0,0,0,1,0,400 mg 4 doses,400 mg,16.7,0,400 mg 4 doses,400 mg
134,75,54.10,19.398329,74.000421,2.1,0,0,1,0,0,0,0,1,0,600 mg 3 doses,0 mg,20.3,0,400 mg 3 doses,0 mg
27,77,73.50,27.494880,43.750000,2.8,0,0,0,0,1,0,0,0,1,800 mg 3 doses,300 mg,26,0,600 mg 4 doses,400 mg
82,84,37.50,14.889407,34.722222,2.1,0,0,0,1,0,0,0,1,0,400 mg 3 doses,400 mg,31.1,0,400 mg 4 doses,200 mg


feature importance for loading dose is [0.12938945 0.21663198 0.20687441 0.117007   0.08125968 0.03532456
 0.02732836 0.06609561 0.02736488 0.01929392 0.00530234 0.
 0.03949149 0.02863633]
feature importance for maintenance dose is [0.13598017 0.15115273 0.14043721 0.1899437  0.12161491 0.02509279
 0.02678492 0.10504341 0.03728199 0.01488863 0.00391022 0.
 0.02017274 0.02769657]
